In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from google.colab import files


sns.set(style="whitegrid")


# Upload and import dataset
uploaded = files.upload()
filename = next(iter(uploaded))
df = pd.read_csv(filename)


# Explore dataset
print("First 5 rows:")
print(df.head())


print("\nData info:")
print(df.info())


print("\nColumns:")
print(df.columns.tolist())


# Convert 'Date' column to datetime and sort
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # coerce errors just in case
df = df.sort_values('Date')
df.set_index('Date', inplace=True)


# Check for missing dates or NaT after conversion
if df.index.hasnans:
    print("Warning: Some dates could not be parsed and are missing.")


# Plot historical 'Close Price' trend
plt.figure(figsize=(14,6))
plt.plot(df['Close Price'], label='Close Price')
plt.title('Stock Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()


# Calculate and plot moving averages (20-day and 50-day)
df['MA20'] = df['Close Price'].rolling(window=20).mean()
df['MA50'] = df['Close Price'].rolling(window=50).mean()


plt.figure(figsize=(14,6))
plt.plot(df['Close Price'], label='Close Price')
plt.plot(df['MA20'], label='20-Day Moving Average')
plt.plot(df['MA50'], label='50-Day Moving Average')
plt.title('Stock Closing Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


# Seasonal decomposition (monthly approx., period=30)
result = seasonal_decompose(df['Close Price'], model='additive', period=30)


plt.figure(figsize=(14,10))
result.plot()
plt.suptitle('Seasonal Decomposition of Closing Prices')
plt.show()


# Correlation with 'Total Traded Quantity' (volume)
if 'Total Traded Quantity' in df.columns:
    plt.figure(figsize=(10,6))
    sns.scatterplot(x='Total Traded Quantity', y='Close Price', data=df)
    plt.title('Scatter Plot of Close Price vs Total Traded Quantity')
    plt.xlabel('Total Traded Quantity')
    plt.ylabel('Close Price')
    plt.show()


    corr = df['Close Price'].corr(df['Total Traded Quantity'])
    print(f"Correlation between Close Price and Total Traded Quantity: {corr:.4f}")
else:
    print("No 'Total Traded Quantity' column found for correlation analysis.")


# Forecasting using ARIMA
arima_model = ARIMA(df['Close Price'], order=(5,1,0))
arima_result = arima_model.fit()
print(arima_result.summary())


# Forecast next 30 days
arima_forecast = arima_result.forecast(steps=30)


plt.figure(figsize=(14,6))
plt.plot(df['Close Price'], label='Historical Close Price')
plt.plot(arima_forecast.index, arima_forecast, label='ARIMA Forecast', color='red')
plt.title('ARIMA Forecast of Closing Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


# Holt-Winters Exponential Smoothing forecast
hw_model = ExponentialSmoothing(df['Close Price'], trend='add', seasonal=None, initialization_method='estimated')
hw_fit = hw_model.fit()
hw_forecast = hw_fit.forecast(30)


plt.figure(figsize=(14,6))
plt.plot(df['Close Price'], label='Historical Close Price')
plt.plot(hw_forecast.index, hw_forecast, label='Holt-Winters Forecast', color='green')
plt.title('Holt-Winters Forecast of Closing Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()






